# Noisy Label Detection

In [1]:
#!/usr/bin/python
from __future__ import absolute_import

import numpy as np
from tensorflow import keras
import pickle
import argparse
import copy
import random

from shapley.apps import Label, Poisoning
from shapley.loader import FashionMnist, MNIST
from shapley.measures import KNN_Shapley, KNN_LOO, G_Shapley, LOO, TMC_Shapley
from shapley.utils.plotter import LabelPlotter, PoisoningPlotter

%matplotlib inline

/home/zidaneandmessi/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/zidaneandmessi/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/zidaneandmessi/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/zidaneandmessi/.local/lib/python3.6/site-packages/tensorflow/

##### data loading

In [2]:
num = 1000
#loader = MNIST(num_train=num, one_hot=False, shuffle=True, by_label=True)
loader = FashionMnist(num_train=num)
X_data, y_data, X_test_data, y_test_data = loader.prepare_data()

###### measure

In [3]:
measure_KNN = KNN_Shapley(K=5)
measure_KNNLOO = KNN_LOO(K=5)
# measure = G_Shapley()
# measure = TMC_Shapley()
# measure = LOO()

##### application

In [4]:
app = Label(X_data, y_data, X_test_data, y_test_data, model_family='NN')

##### run

In [ ]:
res_knn = app.run(measure_KNN)
res_knnloo = app.run(measure_KNNLOO)

done!
result shown below:
[ 8.60734147e-04 -5.79108828e-05  2.15681489e-03  2.67265484e-03
  7.03794200e-04  8.70775481e-04  9.47779086e-04  7.14860328e-04
  7.81089740e-04  1.33701227e-03  9.15931702e-04  9.92651015e-04
 -2.53988069e-04  1.70295834e-04  6.52692355e-04  3.81503812e-04
  1.76740120e-03  1.66151264e-03  4.37112306e-04  9.30548961e-04
  1.05040909e-03  9.89538892e-04  9.63497059e-04  9.01085275e-04
  9.72782226e-04  1.24672884e-03  1.41182795e-03  1.13217644e-03
  9.82147979e-04 -6.87114429e-04  9.91828578e-04  9.14968124e-04
  9.84957727e-04  8.28768544e-04  1.97453305e-03  1.31430784e-03
  1.61432589e-03 -8.02455583e-04  1.07930850e-03  7.89517677e-04
  5.84573732e-04  1.70033593e-03  8.00805749e-04  8.75307147e-04
  1.13234849e-03  7.48626376e-04  1.10150593e-03  1.29636133e-03
  1.87529977e-03  1.37131321e-03  1.21718608e-03  1.63807981e-03
  1.34076703e-03  1.53712941e-03 -2.44226291e-03  1.19576201e-03
  8.03735201e-04  3.79632705e-04  1.68570069e-03  1.64706549e-03

##### visualize

In [ ]:
LabelPlotter(app, ('KNN-Shapley',res_knn), ('KNN-LOO',res_knnloo)).plot()

# Pattern-based Watermark Removal

##### data loading

In [ ]:
num = 1000
loader = FashionMnist(num_train=num)
X_data, y_data, X_test_data, y_test_data = loader.prepare_data()

###### measure

In [ ]:
measure_KNN = KNN_Shapley(K=5)
measure_KNNLOO = KNN_LOO(K=5)

##### application

In [ ]:
app = Poisoning(X_data, y_data, X_test_data, y_test_data, model_family='logistic')

##### run

In [ ]:
res_knn = app.run(measure_KNN)
res_knnloo = app.run(measure_KNNLOO)

##### visualize

In [ ]:
PoisoningPlotter(app, ('KNN-Shapley',res_knn), ('KNN-LOO',res_knnloo)).plot()

# Instance-based Watermark Removal

##### data loading

In [ ]:
# load MNIST

###### measure

In [ ]:
measure = KNN_Shapley(K=5)

##### application

In [ ]:
app = # instance-based watermark removal

##### run

In [ ]:
# run

##### visualize

In [ ]:
# TO BE COMPLETED

# Data Summarization

##### data loading

In [ ]:
# load UCI Adult dataset

###### measure

In [ ]:
measure = KNN_Shapley(K=5)

##### application

In [ ]:
app = # data summarization

##### run

In [ ]:
# run

##### visualize

In [ ]:
# TO BE COMPLETED

# Data Acquisition

##### data loading

In [ ]:
# load MNIST

###### measure

In [ ]:
measure = KNN_Shapley(K=5)

##### application

In [ ]:
app = # data acquisition

##### run

In [ ]:
# run

##### visualize

In [ ]:
# TO BE COMPLETED

# Domain Adapatation

##### data loading

In [ ]:
# load two datasets

###### measure

In [ ]:
measure = KNN_Shapley(K=5)

##### application

In [ ]:
app = # domain adaptation

##### run

In [ ]:
# run

##### visualize

In [ ]:
# TO BE COMPLETED